**Hidden Markov models for cracking codes**

In this exercise you have to make a partially built HMM work and use it to solve some simple substitution ciphers. Plaintext data is provided in 'plaintext' directory. Encrypted data is in 'encrypted'. Some of the texts were originally English some of them were Russian; the sequences are also of different lengths. 

This homework is worth **15 points** and is due by the next class (**24th Oct.**), please submit the results of the **TASK 5** (a list of files and names of the author/work) to Anytask in the following format: 'filename author' where 'filename' is a file from "encrypted/\*_encrypted.txt" and 'author' is a file from "plaintext/\*.txt" (not including 'english.txt', 'russian.txt' or 'all.txt') which best matches the decrypted text.




In [5]:
# Utilities for loading data from file and converting characters to integers and back.
import numpy as np
    
def get_char_to_int_mapping(path):
    # Load data from path and get mapping from characters to integers and back.
    characters = set()
    for line in open(path):
        characters.update(set([c for c in line.strip()]))
    char_to_int_mapping = dict([(char, i) for i, char in enumerate(sorted(list(characters)))])
    int_to_char_mapping = [char for char, i in char_to_int_mapping.items()]
    return char_to_int_mapping, int_to_char_mapping

def load_sequences(path, char_to_int_mapping):
    # Load data from path and map to integers using mapping.
    return [[char_to_int_mapping[c] for c in line.strip()] for line in open(path)]

def estimate_markov_model_from_sequences(sequences, num_states):
    # Estimate a Markov model based on the sequences (integers) provided.

    # pi[i] = Pr(s_0 = i)
    pi_counts = np.zeros(num_states)

    # A[i, j] = Pr(s_t = j | s_{t-1} = i)
    A_counts = np.zeros((num_states, num_states))

    for n, sequence in enumerate(sequences):
        # assert False, "Collect counts for pi and A and return parameter estimates."
        if sequence:
            pi_counts[sequence[0]] += 1
            for i in range(1, len(sequence)):
                A_counts[sequence[i - 1], sequence[i]] += 1
    pi = pi_counts / np.sum(pi_counts)
    A = A_counts / (np.sum(A_counts, axis=1, keepdims=True) + 1e-9)
    
    return pi, A

**TASK 1**: Make the following block run by completing the method 'estimate_markov_model_from_sequences' above.

In [152]:
# Some data to use.
plaintext = 'plaintext/english.txt'
# plaintext = 'plaintext/shakespeare.txt'
# plaintext = 'plaintext/russian.txt'

ciphertext = 'encrypted/1_encrypted.txt' # short sequences in english
# ciphertext = 'encrypted/99_encrypted.txt' # longer sequences in russian

# load a character to integer mapping and reverse                                                                                                         
char_to_int_mapping, int_to_char_mapping = get_char_to_int_mapping(plaintext)

# load sequences as ints                                                                                                                                  
plaintext_sequences = load_sequences(plaintext, char_to_int_mapping)
encrypted_sequences = load_sequences(ciphertext, char_to_int_mapping)

# estimate a markov model over characters
test_pi, test_A = estimate_markov_model_from_sequences([[0, 1], [1, 0, 1], [0, 0, 2]], 3)
assert np.allclose(test_pi, [2 / 3, 1 / 3, 0])
assert np.allclose(test_A, [[1 / 4, 2 / 4, 1 / 4],
                            [1, 0, 0],
                            [0, 0, 0]])

pi, A = estimate_markov_model_from_sequences(plaintext_sequences, len(char_to_int_mapping))

assert np.allclose(np.sum(pi), 1)
assert np.allclose(np.sum(A, axis=1), np.ones((len(char_to_int_mapping))))

Below is a mostly implemented HMM.

In [153]:
class HMM():

    def __init__(self, observations_to_char_mapping={}, states_to_char_mapping={}):
        # Determine number of states and observation space. 
        self.num_states = len(states_to_char_mapping)
        self.num_outputs = len(observations_to_char_mapping)
        self.states_to_char_mapping = states_to_char_mapping
        self.observations_to_char_mapping = observations_to_char_mapping
       
        # Random initialization
        self.pi = np.random.rand(self.num_states)
        self.pi /= np.sum(self.pi)
        self.A = np.random.rand(self.num_states, self.num_states)
        self.A /= np.sum(self.A, 1, keepdims=True)
        self.B = np.random.rand(self.num_states, self.num_outputs)
        self.B /= np.sum(self.B, 1, keepdims=True) 
        
    def estimate_with_em(self, sequences, parameters={}, epsilon=0.001, max_iters=100):
        # Estimates all parameters not provided in 'parameters' based on 'sequences'.
        self.fixed_pi = 'pi' in parameters
        if self.fixed_pi:
            self.pi = parameters['pi']
        self.fixed_A = 'A' in parameters
        if self.fixed_A:
            self.A = parameters['A']
        self.fixed_B = 'B' in parameters
        if self.fixed_B:
            self.B = parameters['B']
    
        previous_llh = None
        iter = 0
        while True and iter < max_iters:
            # Infer expected counts.
            pi_counts, A_counts, B_counts, log_likelihood = self.e_step(sequences)

            # Update parameters based on counts.
            self.m_step(pi_counts, A_counts, B_counts)

            # Output some sequences for debugging.
            self.output(sequences[:10])

            # Log likelihood should be increasing
            print('iteration %d; log likelihood %.4f' % (iter, log_likelihood))
            if previous_llh:
                assert log_likelihood >= previous_llh
                if log_likelihood - previous_llh < epsilon:
                    break
            previous_llh = log_likelihood
        
            iter += 1


    def e_step(self, sequences):
        # Reset counters of statistics
        pi_counts = np.zeros_like(self.pi)
        A_counts = np.zeros_like(self.A) 
        B_counts = np.zeros_like(self.B) 
        total_log_likelihood = 0.0

        for sequence in sequences:
            # Run Forward-Backward dynamic program
            alpha, beta, gamma, xi, log_likelihood = self.forward_backward(sequence)
  
            # Accumulate statistics.
            pi_counts += gamma[:, 0]
            A_counts += xi
            for t, x in enumerate(sequence):
                B_counts[:, x] += gamma[:, t]
            
            total_log_likelihood += log_likelihood

        return pi_counts, A_counts, B_counts, total_log_likelihood

    def m_step(self, pi_counts, A_counts, B_counts):
        if not self.fixed_pi:
            self.pi = pi_counts / np.sum(pi_counts)
        if not self.fixed_A:
            self.A = A_counts / np.sum(A_counts, 1, keepdims=True)
        if not self.fixed_B:
            self.B = B_counts / np.sum(B_counts, 1, keepdims=True)
        
    def max_posterior_decode(self, sequence):
        _, _, gamma, _, log_likelihood = self.forward_backward(sequence)
        return np.argmax(gamma, 0)
        
    def forward_backward(self, sequence):
        # alpha[i][t] = p(x_1, ..., x_t, z_t = i)
        alpha = self.forward(sequence)
        
        # beta[i][t] = p(x_t+1, ..., x_T|z_t = i)
        beta = self.backward(sequence)

        # gamma[i][t] = p(z_t = i|x_1, ..., x_T)
        gamma = (alpha * beta) / np.sum(alpha * beta, 0)

        # xi[i][j] = p(z_t = i, z_{t+1} = j|x_1, ..., x_T)
        xi = np.zeros_like(self.A)
        for t in range(1, len(sequence)-1):
            this_xi = np.zeros_like(self.A)
            for i in range(self.num_states):
                for j in range(self.num_states):
                    this_xi[i, j] += alpha[i, t] * self.A[i, j] * beta[j, t+1] * self.B[j, sequence[t+1]]        
            xi += this_xi / np.sum(this_xi)
                
        return alpha, beta, gamma, xi, np.log(np.sum(alpha[:, len(sequence)-1]))

    def forward(self, sequence):
        # alpha[i][t] = p(x_1, ..., x_t, z_t = i)
        alpha = np.zeros((len(self.pi), len(sequence)))
        for i in range(len(self.pi)):
            alpha[i, 0] = self.pi[i] * self.B[i, sequence[0]]
        for i in range(1, len(sequence)):
            for j in range(len(self.pi)):
                for k in range(len(self.pi)):
                    alpha[k, i] += alpha[j, i - 1] * self.A[j, k] * self.B[k, sequence[i]]
        return alpha 
    
    def backward(self, sequence):
        # beta[i][t] = p(x_t+1, ..., x_T|z_t = i)
        beta = np.zeros((len(self.pi), len(sequence)))
        for i in range(len(self.pi)):
            beta[i, len(sequence) - 1] = 1
        for i in range(len(sequence) - 2, -1, -1):
            for j in range(len(self.pi)):
                for k in range(len(self.pi)):
                    beta[k, i] += beta[j, i + 1] * self.A[k, j] * self.B[j, sequence[i + 1]]
        return beta

    def output(self, sequences):
        # Output some decoded states. 
        for i, sequence in enumerate(sequences):
            observations = [self.observations_to_char_mapping[x] for x in sequence]                
            map_states = [self.states_to_char_mapping[x] for x in self.max_posterior_decode(sequence)]
            print('(states):       %s\n(observations): %s' % (''.join(map_states), ''.join(observations)))


**TASK 2**: Implement the assertions in 'forward' and 'backward' methods on the HMM class so that the following block passes.

In [155]:
# Since it's a substitution cipher we assume hidden states and observations have same alphabet.
state_to_char_mapping = int_to_char_mapping
observation_to_char_mapping = int_to_char_mapping

# Initialize a HMM with the correct state/output spaces.
hmm = HMM(observation_to_char_mapping, state_to_char_mapping)

# Estimate the parameters and decode the encrypted sequences.
hmm.estimate_with_em(encrypted_sequences[:100], parameters={'pi': pi, 'A': A})

(states):       t  t   t    n   t
(observations): noeixjtcoxexhwfei
(states):       t   t    e t     re h     
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       t   h    on e   e  e  r n       e   n
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       t    n       o r         o  e  
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       s s n  o te  e        e t  o    e   n
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       t    n    tn  r t 
(observations): nwxfeaqxcwi gngwij
(states):       be t      e t e t n  h       t  
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       t   t   r   
(observations): cejjgtjxgxef
(states):       be t   h    t e t n  h  
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       t   t    e      r    n  h       r n  e      n
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 0; log likelihood -12136.7277
(states):       t  r   t    n   t
(observations): noeixjtcoxex

(states):       qund  d ange th me tend s thill vengot the ll
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 8; log likelihood -9805.6749
(states):       thir s th u nefor
(observations): noeixjtcoxexhwfei
(states):       qund  d be t s bu t met to
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       all met ory nd at ver vengot veredoll
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       te henge th ar s it u sely on a
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       ilyon ar teitt th ublon thir veredoll
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       te tind bery therd
(observations): nwxfeaqxcwi gngwij
(states):       be t s me te her ind met bund  s
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       qund  d s it
(observations): cejjgtjxgxef
(states):       be t s s sor ver ind met
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       qund  d ange th me tend s thill wengot thyoll
(

(states):       br t s ge te her are met wissans
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       qussand f if
(observations): cejjgtjxgxef
(states):       br t s f sar her are met
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       qussand ange we me were f thill bengot crsoll
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 17; log likelihood -9116.1145
(states):       thin s th i refin
(observations): noeixjtcoxexhwfei
(states):       qussand br t s buit met fo
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       all met ond re at her bengot herrsoll
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       te henge we an f if i selyoor a
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       elyor an printawe ablor thin herrsoll
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       te wind wendithend
(observations): nwxfeaqxcwi gngwij
(states):       br t s ge te her are met wissans
(observations

(states):       br t s ge te her are met wussins
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       qussins y am
(observations): cejjgtjxgxef
(states):       br t s y say her are met
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       qussins ange me me mere y thall bergot mysell
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 26; log likelihood -8904.5669
(states):       than s ch a reman
(observations): noeixjtcoxexhwfei
(states):       qussins br t s bait met me
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ill met ondare it yer bergot yeursell
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       te hedge me in i am a seldior i
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       aldor in practice ablor than yeursell
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       te mave wendithend
(observations): nwxfeaqxcwi gngwij
(states):       br t s ge te yer are met wussins
(observations

(states):       brutus go to you are not wassous
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       cassous i am
(observations): cejjgtjxgxef
(states):       brutus i say you are not
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       cassous urge me no more i thall forget myself
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 35; log likelihood -8642.0269
(states):       than such a roman
(observations): noeixjtcoxexhwfei
(states):       cassous brutus bait not me
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ill not endure it you forget yourself
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       to hedge me in i am a soldier i
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       older in prackice abler than yourself
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       to make conditions
(observations): nwxfeaqxcwi gngwij
(states):       brutus go to you are not wassous
(observations

...

(states):       brutus go to you are not cassous
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       cassous i am
(observations): cejjgtjxgxef
(states):       brutus i say you are not
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       cassous inge me no more i thall forget myself
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 62; log likelihood -8558.8806
(states):       than such a roman
(observations): noeixjtcoxexhwfei
(states):       cassous brutus bait not me
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ill not endure it you forget yourself
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       to hedge me in i am a soldier i
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       older in prackice abler than yourself
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       to mave conditions
(observations): nwxfeaqxcwi gngwij
(states):       brutus go to you are not cassous
(observations

(states):       brutus go to you are not cassous
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       cassous i am
(observations): cejjgtjxgxef
(states):       brutus i say you are not
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       cassous inge me no more i thall forget myself
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 71; log likelihood -8556.3608
(states):       than such a roman
(observations): noeixjtcoxexhwfei
(states):       cassous brutus bait not me
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ill not endure it you forget yourself
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       to hedge me in i am a soldier i
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       older in prackice abler than yourself
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       to mave conditions
(observations): nwxfeaqxcwi gngwij
(states):       brutus go to you are not cassous
(observations

(states):       cassous inge me no more i thall forget myself
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 80; log likelihood -8555.0037
(states):       than such a roman
(observations): noeixjtcoxexhwfei
(states):       cassous brutus bait not me
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ill not endure it you forget yourself
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       to hedge me in i am a soldier i
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       older in prackice abler than yourself
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       to mave conditions
(observations): nwxfeaqxcwi gngwij
(states):       brutus go to you are not cassous
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       cassous i am
(observations): cejjgtjxgxef
(states):       brutus i say you are not
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       cassous inge me no more i thall forget myself


(states):       brutus go to you are not cassous
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       cassous i am
(observations): cejjgtjxgxef
(states):       brutus i say you are not
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       cassous inge me no more i thall fonget myself
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 89; log likelihood -8554.3345
(states):       than such a roman
(observations): noeixjtcoxexhwfei
(states):       cassous brutus bait not me
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ill not endure it you fonget yourself
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       to hedge me in i am a soldier i
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       older in prackice abler than yourself
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       to mave conditions
(observations): nwxfeaqxcwi gngwij
(states):       brutus go to you are not cassous
(observations

(states):       cassous inge me no more i thall fonget myself
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 98; log likelihood -8553.9002
(states):       than such a roman
(observations): noeixjtcoxexhwfei
(states):       cassous brutus bait not me
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ill not endure it you fonget yourself
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       to hedge me in i am a soldier i
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       older in prackice abler than yourself
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       to mave conditions
(observations): nwxfeaqxcwi gngwij
(states):       brutus go to you are not cassous
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       cassous i am
(observations): cejjgtjxgxef
(states):       brutus i say you are not
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       cassous inge me no more i thall fonget myself


**TASK 3**: Some of the encrypted sequences are quite long. Try decoding some from 'encrypted/99_encrypted.txt' (note these are in Russian).

In [156]:
# Some data to use.
plaintext = 'plaintext/russian.txt'
ciphertext = 'encrypted/99_encrypted.txt' # longer sequences in russian                                                                                                     
char_to_int_mapping, int_to_char_mapping = get_char_to_int_mapping(plaintext)                                                                                                                                
plaintext_sequences = load_sequences(plaintext, char_to_int_mapping)
encrypted_sequences = load_sequences(ciphertext, char_to_int_mapping)
pi, A = estimate_markov_model_from_sequences(plaintext_sequences, len(char_to_int_mapping))

In [178]:
class HMM():

    def __init__(self, observations_to_char_mapping={}, states_to_char_mapping={}):
        # Determine number of states and observation space. 
        self.num_states = len(states_to_char_mapping)
        self.num_outputs = len(observations_to_char_mapping)
        self.states_to_char_mapping = states_to_char_mapping
        self.observations_to_char_mapping = observations_to_char_mapping
       
        # Random initialization
        self.pi = np.random.rand(self.num_states)
        self.pi /= np.sum(self.pi)
        self.A = np.random.rand(self.num_states, self.num_states)
        self.A /= np.sum(self.A, 1, keepdims=True)
        self.B = np.random.rand(self.num_states, self.num_outputs)
        self.B /= np.sum(self.B, 1, keepdims=True) 
        
    def estimate_with_em(self, sequences, parameters={}, epsilon=0.001, max_iters=100):
        # Estimates all parameters not provided in 'parameters' based on 'sequences'.
        self.fixed_pi = 'pi' in parameters
        if self.fixed_pi:
            self.pi = parameters['pi']
        self.fixed_A = 'A' in parameters
        if self.fixed_A:
            self.A = parameters['A']
        self.fixed_B = 'B' in parameters
        if self.fixed_B:
            self.B = parameters['B']
    
        previous_llh = None
        iter = 0
        while True and iter < max_iters:
            # Infer expected counts.
            pi_counts, A_counts, B_counts, log_likelihood = self.e_step(sequences)

            # Update parameters based on counts.
            self.m_step(pi_counts, A_counts, B_counts)

            # Output some sequences for debugging.
            self.output(sequences[:10])

            # Log likelihood should be increasing
            print('iteration %d; log likelihood %.4f' % (iter, log_likelihood))
            if previous_llh:
                assert log_likelihood >= previous_llh
                if log_likelihood - previous_llh < epsilon:
                    break
            previous_llh = log_likelihood
        
            iter += 1


    def e_step(self, sequences):
        # Reset counters of statistics
        pi_counts = np.zeros_like(self.pi)
        A_counts = np.zeros_like(self.A) 
        B_counts = np.zeros_like(self.B) 
        total_log_likelihood = 0.0

        for sequence in sequences:
            # Run Forward-Backward dynamic program
            alpha, beta, gamma, xi, log_likelihood = self.forward_backward(sequence)
  
            # Accumulate statistics.
            pi_counts += gamma[:, 0]
            A_counts += xi
            for t, x in enumerate(sequence):
                B_counts[:, x] += gamma[:, t]
            
            total_log_likelihood += log_likelihood

        return pi_counts, A_counts, B_counts, total_log_likelihood

    def m_step(self, pi_counts, A_counts, B_counts):
        if not self.fixed_pi:
            self.pi = pi_counts / np.sum(pi_counts)
        if not self.fixed_A:
            self.A = A_counts / np.sum(A_counts, 1, keepdims=True)
        if not self.fixed_B:
            self.B = B_counts / np.sum(B_counts, 1, keepdims=True)
        
    def max_posterior_decode(self, sequence):
        _, _, gamma, _, log_likelihood = self.forward_backward(sequence)
        return np.argmax(gamma, 0)
        
    def forward_backward(self, sequence):
        # alpha[i][t] = p(x_1, ..., x_t, z_t = i)
        alpha, norm = self.forward(sequence)
        
        # beta[i][t] = p(x_t+1, ..., x_T|z_t = i)
        beta = self.backward(sequence)

        # gamma[i][t] = p(z_t = i|x_1, ..., x_T)
        gamma = (alpha * beta) / np.sum(alpha * beta, 0)

        # xi[i][j] = p(z_t = i, z_{t+1} = j|x_1, ..., x_T)
        xi = np.zeros_like(self.A)
        for t in range(1, len(sequence)-1):
            this_xi = np.zeros_like(self.A)
            for i in range(self.num_states):
                for j in range(self.num_states):
                    this_xi[i, j] += alpha[i, t] * self.A[i, j] * beta[j, t+1] * self.B[j, sequence[t+1]]        
            xi += this_xi / np.sum(this_xi)
                
        return alpha, beta, gamma, xi, np.log(np.sum(alpha[:, len(sequence)-1])) + norm

    def forward(self, sequence):
        # alpha[i][t] = p(x_1, ..., x_t, z_t = i)
        alpha = np.zeros((len(self.pi), len(sequence)))
        for i in range(len(self.pi)):
            alpha[i, 0] = self.pi[i] * self.B[i, sequence[0]]
        norm = np.log(np.max(alpha[:, 0]))
        alpha[:, 0] /= np.max(alpha[:, 0])
        for i in range(1, len(sequence)):
            for j in range(len(self.pi)):
                for k in range(len(self.pi)):
                    alpha[k, i] += alpha[j, i - 1] * self.A[j, k] * self.B[k, sequence[i]]
            norm += np.log(np.max(alpha[:, i]))
            alpha[:, i] /= np.max(alpha[:, i])
        return alpha, norm
    
    def backward(self, sequence):
        # beta[i][t] = p(x_t+1, ..., x_T|z_t = i)
        beta = np.zeros((len(self.pi), len(sequence)))
        for i in range(len(self.pi)):
            beta[i, len(sequence) - 1] = 1
        for i in range(len(sequence) - 2, -1, -1):
            for j in range(len(self.pi)):
                for k in range(len(self.pi)):
                    beta[k, i] += beta[j, i + 1] * self.A[k, j] * self.B[j, sequence[i + 1]]
            beta[:, i] /= np.max(beta[:, i])
        return beta

    def output(self, sequences):
        # Output some decoded states. 
        for i, sequence in enumerate(sequences):
            observations = [self.observations_to_char_mapping[x] for x in sequence]                
            map_states = [self.states_to_char_mapping[x] for x in self.max_posterior_decode(sequence)]
            print('(states):       %s\n(observations): %s' % (''.join(map_states), ''.join(observations)))

In [179]:
# Some data to use.
plaintext = 'plaintext/english.txt'
ciphertext = 'encrypted/1_encrypted.txt' # longer sequences in russian                                                                                                     
char_to_int_mapping, int_to_char_mapping = get_char_to_int_mapping(plaintext)                                                                                                                                
plaintext_sequences = load_sequences(plaintext, char_to_int_mapping)
encrypted_sequences = load_sequences(ciphertext, char_to_int_mapping)
pi, A = estimate_markov_model_from_sequences(plaintext_sequences, len(char_to_int_mapping))
state_to_char_mapping = int_to_char_mapping
observation_to_char_mapping = int_to_char_mapping
hmm = HMM(observation_to_char_mapping, state_to_char_mapping)
hmm.estimate_with_em(encrypted_sequences[:10], parameters={'pi': pi, 'A': A})

(states):       t t  s    a  s t 
(observations): noeixjtcoxexhwfei
(states):       ahii  i    e i  o ee  e  e
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       t  e  e e t  e  he se e   ee     re e
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       t t e  e  e      t  t i  e e n 
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       s ee     e t e  e t  e te t      re e
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       t e the  h   e s r
(observations): nwxfeaqxcwi gngwij
(states):       wo e i    e   se t e   e  tii  i
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       ahii  i   t 
(observations): cejjgtjxgxef
(states):       wo e i   it t se t e   e
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       ahii  it   e  e   e   e   s t   e   ee   re e
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 0; log likelihood -934.7088
(states):       t t  o t  a  s t 
(observations): noeixjtcoxexhw

(states):       chin his aver my my when ro thenofovean  nehi
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 8; log likelihood -717.6822
(states):       a t nd burdrof t 
(observations): noeixjtcoxexhwfei
(states):       chin hincouthinch ar man m
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       thes mane thoen an werofovean whanehi
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       amy maver en in nd soriths ean 
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       whaean indouce berotheano t s whanehi
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       amy then win e w n
(observations): nwxfeaqxcwi gngwij
(states):       couthin myomy menther manchin hi
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       chin hin nd 
(observations): cejjgtjxgxef
(states):       couthin rit s menther ma
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       chin his aver my my when ro thenofovean  nehi
(o

(states):       couthin rid s wuntour ma
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       ckinghis aver er my wour sinther foveand ithe
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 17; log likelihood -684.9197
(states):       and se burdyof t 
(observations): noeixjtcoxexhwfei
(states):       ckinghincouthinck ar man e
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ther mare thour ar wuy fovean whanthe
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       amy maver er in nd sdrithe man 
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       whaton indouca beruchearand s whanthe
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       amy ther wis t w i
(observations): nwxfeaqxcwi gngwij
(states):       couthis myomy wuntour manckinghi
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       ckinghin nd 
(observations): cejjgtjxgxef
(states):       couthin rid s wuntour ma
(observations): khtntj

(states):       couthin rid s wundour ma
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       ckin hishaver ur my four sinther foveand ithe
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 26; log likelihood -677.3873
(states):       and se burdyof d 
(observations): noeixjtcoxexhwfei
(states):       ckin hincouthinck ar man t
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ther mare thour ar wur fovean whanthe
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       amy mever ur in nd ndrithe ton 
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       whepon ingou t berd hearand s whanthe
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       amy ther wit t wis
(observations): nwxfeaqxcwi gngwij
(states):       couthis bramy wundour manckin he
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       ckin hin nd 
(observations): cejjgtjxgxef
(states):       couthin rid s wundour ma
(observations): khtntj

(states):       couthes bramy wundour manckin he
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       ckin hin nd 
(observations): cejjgtjxgxef
(states):       couthin red s wundour ma
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       ckin hishaverour myofour sinther foveano ithe
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 35; log likelihood -675.5409
(states):       and se qurdyofod 
(observations): noeixjtcoxexhwfei
(states):       ckin hincouthen d ar manou
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ther mare thour ar wur fovean whanthe
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       amy meverour in ndondrithe pon 
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       whepan ingod t bund hearand s whanthe
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       amy ther wit t wis
(observations): nwxfeaqxcwi gngwij
(states):       couthes bramy wundour manckin he
(observations)

...

(states):       ckis heshaverour myofour sinther moveano ithe
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 62; log likelihood -674.4402
(states):       and se condyofod 
(observations): noeixjtcoxexhwfei
(states):       ckis hincouthen d ar manou
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ther mare thour an wur movean whasthe
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       amyoueverour in ndondrithe pon 
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       weepan ingod t cund hearand s whasthe
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       amyother wit t wis
(observations): nwxfeaqxcwi gngwij
(states):       couthes bramy windour manckis he
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       ckis hen ndo
(observations): cejjgtjxgxef
(states):       couthen red s windour ma
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       ckis heshaverour myofour sinther moveano ithe
(

(states):       ckistheshaverour myofour lingher maveano ithe
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 71; log likelihood -673.4381
(states):       and ss condyofod 
(observations): noeixjtcoxexhwfei
(states):       ckisthincouthen d an manou
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ther mare thour an bur mavean whasthe
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       amyoueverour in ndondrithe pon 
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       wexpan ingod t cund hearand s whasthe
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       amyother wit t wis
(observations): nwxfeaqxcwi gngwij
(states):       couthen bramy windour manckisthe
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       ckisthen ndo
(observations): cejjgtjxgxef
(states):       couthen led y windour ma
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       ckistheshaverour myofour lingher maveano ithe
(

(observations): cejjgtjxgxef
(states):       fouthen led y windour ma
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       ckistheshaverour myofour lingher maveano ithe
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 80; log likelihood -672.9572
(states):       and ss condyofod 
(observations): noeixjtcoxexhwfei
(states):       ckisthencouthen d an manou
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ther mare thour an bur mavean whasthe
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       amyoueverour in ndondrithe por 
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       wexpan ingod t cund hearand s whasthe
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       amyother wit t wis
(observations): nwxfeaqxcwi gngwij
(states):       fouthen bramy windour manckisthe
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       ckisthen ndo
(observations): cejjgtjxgxef
(states):       fouthen led y wind

(states):       fouthen led y windour ma
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       ckistheshaverour myofour lingher maveano ithe
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 89; log likelihood -672.8385
(states):       and ss condrofod 
(observations): noeixjtcoxexhwfei
(states):       ckisthen outhen d an manou
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ther mare thour an bur mavean whasthe
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       amyoueverour in ndondrithe tor 
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       wexpan ingod t cund hearand s whasthe
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       amyother wit t wis
(observations): nwxfeaqxcwi gngwij
(states):       fouthen bramy windour manckisthe
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       ckisthen ndo
(observations): cejjgtjxgxef
(states):       fouthen led y windour ma
(observations): khtntj

(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       ckisthen ndo
(observations): cejjgtjxgxef
(states):       fouthen led y windour ma
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       ckistheshaverour myofour lingher maveano ithe
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 98; log likelihood -672.7426
(states):       and seucondrofod 
(observations): noeixjtcoxexhwfei
(states):       ckisthen outhen d an manou
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ther mare thour an bur mavean whasthe
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       amyoueverour in ndondr the tor 
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       whepan ingod t cund hearand s whasthe
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       amy ther wit t wis
(observations): nwxfeaqxcwi gngwij
(states):       fouther bramy windour manckisthe
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):     

In [158]:
state_to_char_mapping = int_to_char_mapping
observation_to_char_mapping = int_to_char_mapping
hmm = HMM(observation_to_char_mapping, state_to_char_mapping)
hmm.estimate_with_em(encrypted_sequences[:1], parameters={'pi': pi, 'A': A})

(states):       не   а е  о а  о оа оо   оала а    ооо а е а а    о ао  о оооа в о оат оо   ао о  о в ае  на ооа не оела о то оооа  т а  т  а а а а   е а е а т   а   о а        а и   оооа  т а  т  а  а и оа   ооа ооа    аоа   о    а и оала а а а   е а   ооа ооа    а  т а  т  а аоо  ала а о  оее аео  о  ее а о е е   а е  о а ооа   о   ооаоа   е а   ооа ооа    о а   оа   о   о е е   а е  о а оо а  о о е аеооа   ае а о а  в а не оеа    оао а   еоо оо  оее а   а ела о   ооа    оао а елалаеоа  о   о   ооа   ола а о аое  оа а о  о  о ао  а  о  еоо
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяръуо0ъпф о5мфювпъуошщн1ошщн1повоф 5юо о0ювмщгмддюодмю4 шпддюошфройдрьропдшщмроюхмоьпяфпйпф о оюяръуо0ъпф о5мфювпъу0роъю4моьпяфпйпфпойдрьцопдшщмцох2фюою3мв шдюодмфювйюодюошфрошвнто4мд8 дойпьпфю0уоъпйом0ъм0ъ

(states):       14007194 стра м пой огль дала в вство на пра встпо нагрологола я к лая ость и понел тра мотам та бе векамо т олола стьи стька вом им нена праст ома вненам ва пональ полола стьи стька паст вай пола пой поналамоск понаст лала вом им ненай пола пой пона стьи стька и л мала в остое ма ом в пе ма 19214000194 страм та к 1919191925 ненай пола пой понени м на к 191919214007194 стра комам о л ема гом  на ма она стьа бе в е к пом на прато гло те маломамела 1919192191916 намелала на сть вов полалом ка вромалекона в остроненаломай в пело
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяръуо0ъпф о5мфювпъуошщн1ошщн1повоф 5юо о0ювмщгмддюодмю4 шпддюошфройдрьропдшщмроюхмоьпяфпйпф о оюяръуо0ъпф о5мфювпъу0роъю4моьпяфпйпфпойдрьцопдшщмцох2фюою3мв шдюодмфювйюодюошфрошвнто4мд8 дойпьпфю0уоъпйом0ъм0ъ

(states):       14087194 стри в пою огоголали в встло ни пра вс по нафрулеголи т колатрокакая пожелатра вотий би бегл камо трололи стья стька вой ий нени прастьеми внений ва пожаль пололи стья стька кастолалоголи бой пожалимоск пожастолали вой ий нежароголи бой пожи стья стька изл вали в тсете ви тв л по ви 19214080194 стний би к 1919191926 нежароголи бой поженией би к 191919214087194 стра ковимоталь ви бов  ви ви они стья бегл я к пой ни пратробро те вицомим ли 1919192191916 ним лали би сть вогоголицей ка вровал поби в трележенихомалал почт
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяръуо0ъпф о5мфювпъуошщн1ошщн1повоф 5юо о0ювмщгмддюодмю4 шпддюошфройдрьропдшщмроюхмоьпяфпйпф о оюяръуо0ъпф о5мфювпъу0роъю4моьпяфпйпфпойдрьцопдшщмцох2фюою3мв шдюодмфювйюодюошфрошвнто4мд8 дойпьпфю0уоъпйом0ъм0ъ

(states):       14087194 стри в пою отоголали в всело ни при всего назр летоли т колитрокакая пожелатри вотий би бъел камо трололи стья стьки вой ий нужи пристьеви внежий ва пожазь пололи стья стьки кастолароголи пой кожилимоскакожистолали вой ий нужероголи пой кожи стья стька изл вали в прато ви твал ко ви 19254089194 стний би к огоголалий нужероголи пой коженией би к 191919254087194 стри ковивоталь ви пова ви ви они стья бъел и коговежи пратробро то вицомим ли 1919192191916 жим лали би сть вогоголицей ки вровал гоби в пралужениховазал кочт
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяръуо0ъпф о5мфювпъуошщн1ошщн1повоф 5юо о0ювмщгмддюодмю4 шпддюошфройдрьропдшщмроюхмоьпяфпйпф о оюяръуо0ъпф о5мфювпъу0роъю4моьпяфпйпфпойдрьцопдшщмцох2фюою3мв шдюодмфювйюодюошфрошвнто4мд8 дойпьпфю0уоъпйом0ъм0ъ

(states):       14087194 стри в кою отоголили в вселозни при всегозназь дутоли токолитрокакая гожалатья вотий би брел каму трулоли стья стьки вов ий нужи при ть ви внежий во кужезь кололи стья стьки кистолеруголи пой кужелимоскакужестолили вов ий нужеруголи пой кужи стья стьки изл вализв праты ви твал ко ви 19254089194 стний би к огоголилий нужеруголи пой кужени в би к 191919254087194 стья ковивоталь ви пова ви ви они стья брел и коговежи пратрупру то вицувив ли 19191921919о6 жев лили би сть вогоголицой ки вровал гобизв пралужениховазал кочт
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяръуо0ъпф о5мфювпъуошщн1ошщн1повоф 5юо о0ювмщгмддюодмю4 шпддюошфройдрьропдшщмроюхмоьпяфпйпф о оюяръуо0ъпф о5мфювпъу0роъю4моьпяфпйпфпойдрьцопдшщмцох2фюою3мв шдюодмфювйюодюошфрошвнто4мд8 дойпьпфю0уоъпйом0ъм0ъ

...

(states):       14087192 стри в коя стобылили вывселозни при всегазнись мутоли токолитронакая гожалатья ватий пи прел ниду тралоли стья стьки вав ий нужи при ть ви внежей во кужезь калоли стья стьки кистолезаголи пов кужелидускакужестолили вав ий нужераголи пов кужи стья стьки исл велизвапраты ви твыл ка ви 19154089192 стний пи к окоголилий нужераголи пов кужени в би к 191919154087192 стья ковивоталь ви пова ви ви они стья прел и коговежи претрупру пр вицудей ли окоголи коговежей лили пи сть вогоголицуй ни дровел готизвапралужениховезал к чт
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяръуо0ъпф о5мфювпъуошщн1ошщн1повоф 5юо о0ювмщгмддюодмю4 шпддюошфройдрьропдшщмроюхмоьпяфпйпф о оюяръуо0ъпф о5мфювпъу0роъю4моьпяфпйпфпойдрьцопдшщмцох2фюою3мв шдюодмфювйюодюошфрошвнто4мд8 дойпьпфю0уоъпйом0ъм0ъ

(states):       14087192 стри в коя стобылили вывселозни кри всегазнись мутоли тонолитронакая гожалатья ватей пе прел нида тралоли стья стьки вав ий нужи кри ть ви внежей во кужезь калоли стья стьки кистолезаколи пов кужелидаскакужестолили вав ий нужераколи пов кужи стья стьки исл велизвапратр ви твыл ко ви 19194089192 стний пе к огоголилий нужераколи пов кужени в пе к 191919192087192 стья ковихоталь ви пова ви ви они стья прел и коговежи кретрупру пр вицудей ли огоголи коговежей лили пе сть вогоголицай ни дровел готизвапралуженихадезал к чт
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяръуо0ъпф о5мфювпъуошщн1ошщн1повоф 5юо о0ювмщгмддюодмю4 шпддюошфройдрьропдшщмроюхмоьпяфпйпф о оюяръуо0ъпф о5мфювпъу0роъю4моьпяфпйпфпойдрьцопдшщмцох2фюою3мв шдюодмфювйюодюошфрошвнто4мд8 дойпьпфю0уоъпйом0ъм0ъ

(states):       14087192 стри в коя стобылили вывселозни кри всегазнись мутоли товолитронакая кожалатья ватей пе прел нида тралоли стья стьки вав ей нужи кри ть ди днежей во кужезь калоли стья стьки кистолезаколи повакужелидаскакужестолили вав ей нужезаколи повакужи стья стьки исл велизвапратр ви твыл ко ви 19192089192 стний пе к огоголилий нужезаколи повакужени в пе к 191919192087192 стья вовихоталь ви пова ви ви они стья прел и коговежи кретрупра пр вицудей ли огоголи коговежей лили пе сть вогоголицай ни дровел готизвапралуженицадезал кочт
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяръуо0ъпф о5мфювпъуошщн1ошщн1повоф 5юо о0ювмщгмддюодмю4 шпддюошфройдрьропдшщмроюхмоьпяфпйпф о оюяръуо0ъпф о5мфювпъу0роъю4моьпяфпйпфпойдрьцопдшщмцох2фюою3мв шдюодмфювйюодюошфрошвнто4мд8 дойпьпфю0уоъпйом0ъм0ъ

(states):       14087192 стря в коя стобылили вывселозни кри всегазнись мутоли товолитровакая кожалатья ватий пе прел нида тралоли стья стьки вав ей нужи кри ть ди днежей во кужезь калоли стья стьки кистолезаколи повакужелидаскакужестолили вав ей нужезаколи повакужи стья стьки исл велизвапрато ви твыл ко ви 19192089192 стний пе к окоголилий нужезаколи повакужени в пи к 191919192087192 стья вовихоталь ви пова ви ви они стья прел и коговежи кретрупра пр вицудей ли окоголи коговежей лили пе сть вогоголицай ни дровел готизвапралуженицудезал кочт
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяръуо0ъпф о5мфювпъуошщн1ошщн1повоф 5юо о0ювмщгмддюодмю4 шпддюошфройдрьропдшщмроюхмоьпяфпйпф о оюяръуо0ъпф о5мфювпъу0роъю4моьпяфпйпфпойдрьцопдшщмцох2фюою3мв шдюодмфювйюодюошфрошвнто4мд8 дойпьпфю0уоъпйом0ъм0ъ

(states):       14087192 стре в коя стобылили вывселозни кри всегазнись мутоли товолитровакая кожалатья ватий пе прел веда тралоли стья стьки вай ий нужи кри ть ди днежей во кужезь калоли стья стьки кистолезаколи повакужелидаскакужестолили вай ий нужезаколи повакужи стья стьки исл велизвапрато ви твыл ко ви 19192089192 стний пе к окоголилий нужезаколи повакужени в пи к 191919192087192 стья вовихоталь ви пова ви ви они стья прел и коговежи кретрупра по вицудей ли окоголи коговежей лили пе сть вогоголицай ви дровел готизвапралуженицудезал кочт
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяръуо0ъпф о5мфювпъуошщн1ошщн1повоф 5юо о0ювмщгмддюодмю4 шпддюошфройдрьропдшщмроюхмоьпяфпйпф о оюяръуо0ъпф о5мфювпъу0роъю4моьпяфпйпфпойдрьцопдшщмцох2фюою3мв шдюодмфювйюодюошфрошвнто4мд8 дойпьпфю0уоъпйом0ъм0ъ

**TASK 4**: Make your implementation of forward and backward more efficient by removing all but the outermost for-loop.

In [6]:
class HMM():

    def __init__(self, observations_to_char_mapping={}, states_to_char_mapping={}):
        # Determine number of states and observation space. 
        self.num_states = len(states_to_char_mapping)
        self.num_outputs = len(observations_to_char_mapping)
        self.states_to_char_mapping = states_to_char_mapping
        self.observations_to_char_mapping = observations_to_char_mapping
       
        # Random initialization
        self.pi = np.random.rand(self.num_states)
        self.pi /= np.sum(self.pi)
        self.A = np.random.rand(self.num_states, self.num_states)
        self.A /= np.sum(self.A, 1, keepdims=True)
        self.B = np.random.rand(self.num_states, self.num_outputs)
        self.B /= np.sum(self.B, 1, keepdims=True) 
        
    def estimate_with_em(self, sequences, parameters={}, epsilon=0.001, max_iters=100, verbose=True):
        # Estimates all parameters not provided in 'parameters' based on 'sequences'.
        self.fixed_pi = 'pi' in parameters
        if self.fixed_pi:
            self.pi = parameters['pi']
        self.fixed_A = 'A' in parameters
        if self.fixed_A:
            self.A = parameters['A']
        self.fixed_B = 'B' in parameters
        if self.fixed_B:
            self.B = parameters['B']
    
        previous_llh = None
        iter = 0
        while True and iter < max_iters:
            # Infer expected counts.
            pi_counts, A_counts, B_counts, log_likelihood = self.e_step(sequences)

            # Update parameters based on counts.
            self.m_step(pi_counts, A_counts, B_counts)

            if verbose:
                # Output some sequences for debugging.
                self.output(sequences[:10])

                # Log likelihood should be increasing
                print('iteration %d; log likelihood %.4f' % (iter, log_likelihood))
            if previous_llh:
                assert log_likelihood >= previous_llh
                if log_likelihood - previous_llh < epsilon:
                    break
            previous_llh = log_likelihood
        
            iter += 1
            
        return previous_llh


    def e_step(self, sequences):
        # Reset counters of statistics
        pi_counts = np.zeros_like(self.pi)
        A_counts = np.zeros_like(self.A) 
        B_counts = np.zeros_like(self.B) 
        total_log_likelihood = 0.0

        for sequence in sequences:
            # Run Forward-Backward dynamic program
            alpha, beta, gamma, xi, log_likelihood = self.forward_backward(sequence)
  
            # Accumulate statistics.
            pi_counts += gamma[:, 0]
            A_counts += xi
            for t, x in enumerate(sequence):
                B_counts[:, x] += gamma[:, t]
            
            total_log_likelihood += log_likelihood

        return pi_counts, A_counts, B_counts, total_log_likelihood

    def m_step(self, pi_counts, A_counts, B_counts):
        if not self.fixed_pi:
            self.pi = pi_counts / np.sum(pi_counts)
        if not self.fixed_A:
            self.A = A_counts / np.sum(A_counts, 1, keepdims=True)
        if not self.fixed_B:
            self.B = B_counts / np.sum(B_counts, 1, keepdims=True)
        
    def max_posterior_decode(self, sequence):
        _, _, gamma, _, log_likelihood = self.forward_backward(sequence)
        return np.argmax(gamma, 0)
        
    def forward_backward(self, sequence):
        # alpha[i][t] = p(x_1, ..., x_t, z_t = i)
        alpha, norm = self.forward(sequence)
        
        # beta[i][t] = p(x_t+1, ..., x_T|z_t = i)
        beta = self.backward(sequence)

        # gamma[i][t] = p(z_t = i|x_1, ..., x_T)
        gamma = (alpha * beta) / np.sum(alpha * beta, 0)

        # xi[i][j] = p(z_t = i, z_{t+1} = j|x_1, ..., x_T)
        xi = np.zeros_like(self.A)
        for t in range(1, len(sequence)-1):
            this_xi = np.dot(
                alpha[:, t].reshape(self.num_states, 1),
                (beta[:, t + 1] * self.B[:, sequence[t + 1]]).reshape(1, self.num_states)
            ) * self.A     
            xi += this_xi / np.sum(this_xi)
                
        return alpha, beta, gamma, xi, np.log(np.sum(alpha[:, len(sequence)-1])) + norm

    def forward(self, sequence):
        # alpha[i][t] = p(x_1, ..., x_t, z_t = i)
        alpha = np.zeros((len(self.pi), len(sequence)))
        alpha[:, 0] = self.pi * self.B[:, sequence[0]]
        norm = np.log(np.max(alpha[:, 0]))
        alpha[:, 0] /= np.max(alpha[:, 0])
        for i in range(1, len(sequence)):
            alpha[:, i] = np.dot(
                alpha[:, i - 1].reshape(1, self.num_states),
                self.A
            ).reshape(self.num_states) * self.B[:, sequence[i]]
            norm += np.log(np.max(alpha[:, i]))
            alpha[:, i] /= np.max(alpha[:, i])
        return alpha, norm
    
    def backward(self, sequence):
        # beta[i][t] = p(x_t+1, ..., x_T|z_t = i)
        beta = np.zeros((len(self.pi), len(sequence)))
        beta[:, len(sequence) - 1] = 1
        for i in range(len(sequence) - 2, -1, -1):
            beta[:, i] = np.dot(
                self.A,
                (beta[:, i + 1] * self.B[:, sequence[i + 1]]).reshape(self.num_states, 1)
            ).reshape(self.num_states)
            beta[:, i] /= np.max(beta[:, i])
        return beta

    def output(self, sequences):
        # Output some decoded states. 
        for i, sequence in enumerate(sequences):
            observations = [self.observations_to_char_mapping[x] for x in sequence]                
            map_states = [self.states_to_char_mapping[x] for x in self.max_posterior_decode(sequence)]
            print('(states):       %s\n(observations): %s' % (''.join(map_states), ''.join(observations)))

In [186]:
# Some data to use.
plaintext = 'plaintext/english.txt'
ciphertext = 'encrypted/1_encrypted.txt' # longer sequences in russian                                                                                                     
char_to_int_mapping, int_to_char_mapping = get_char_to_int_mapping(plaintext)                                                                                                                                
plaintext_sequences = load_sequences(plaintext, char_to_int_mapping)
encrypted_sequences = load_sequences(ciphertext, char_to_int_mapping)
pi, A = estimate_markov_model_from_sequences(plaintext_sequences, len(char_to_int_mapping))
state_to_char_mapping = int_to_char_mapping
observation_to_char_mapping = int_to_char_mapping
hmm = HMM(observation_to_char_mapping, state_to_char_mapping)
hmm.estimate_with_em(encrypted_sequences, parameters={'pi': pi, 'A': A})

(states):       ae t o e   e    t
(observations): noeixjtcoxexhwfei
(states):       t th  o te e d t te t e   
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       t   t e  to     h     n  h e  r  t  n
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       a     h      t     e  o        
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       a      t     e   e t    ee t  r  t  n
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       a    t  e t  e  th
(observations): nwxfeaqxcwi gngwij
(states):       te e d h  e   a      t e   to  t
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       t th  o     
(observations): cejjgtjxgxef
(states):       te e o   o h  a      t e
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       t th  oe  h     tr  a     he    n  h e   t  n
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 0; log likelihood -123161.0098
(states):       ae t o ee       t
(observations): noeixjtcoxe

(states):       me t d be a tof t
(observations): noeixjtcoxexhwfei
(states):       wandind bund d ba s tor m 
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ath tove th to  s ron fother yound ll
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       mo e the th  t    m   doth on  
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       athan  t py th the bl n me t yound ll
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       mo m te fathis ath
(observations): nwxfeaqxcwi gngwij
(states):       bund d ho mo hon ano tor fandind
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       wandind    t
(observations): cejjgtjxgxef
(states):       bund d   d y hon ano tor
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       wandind nthe to to mono   de th fother thd ll
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 9; log likelihood -100025.2690
(states):       me t d be a tof t
(observations): noeixjtcoxe

(states):       meat duce a tomat
(observations): noeixjtcoxexhwfei
(states):       wandind bunknd bais tor me
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ill tor sthure is you fothes yourdelf
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       ho yethe me it i am a dothier i
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       other it prachice abler meat yourdelf
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       ho make cothisiond
(observations): nwxfeaqxcwi gngwij
(states):       bunknd ho ho you are tor wandind
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       wandind i am
(observations): cejjgtjxgxef
(states):       bunknd i day you are tor
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       wandind nthe me to more i deall fothes mydelf
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 18; log likelihood -95843.1806
(states):       meat duce a tomat
(observations): noeixjtcoxe

(states):       than tuch a roman
(observations): noeixjtcoxexhwfei
(states):       candind brusus bais tos me
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ill tos endure is you forces yourdelf
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       to hedge me in i am a toldier i
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       older in prachice abler than yourdelf
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       to make condisiond
(observations): nwxfeaqxcwi gngwij
(states):       brusus co so you are tos candind
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       candind i am
(observations): cejjgtjxgxef
(states):       brusus i tay you are tos
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       candind urce me to more i thall forces myself
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 27; log likelihood -90460.5264
(states):       than tuch a roman
(observations): noeixjtcoxe

(states):       than such a roman
(observations): noeixjtcoxexhwfei
(states):       cassous brutus bait not me
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ill not endure it you forget yourself
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       to hedge me in i am a soldier i
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       older in practice abler than yourself
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       to make conditions
(observations): nwxfeaqxcwi gngwij
(states):       brutus go to you are not cassous
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       cassous i am
(observations): cejjgtjxgxef
(states):       brutus i say you are not
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       cassous urge me no more i thall forget myself
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 36; log likelihood -87661.4391
(states):       than such a roman
(observations): noeixjtcoxe

...

(states):       than such a roman
(observations): noeixjtcoxexhwfei
(states):       cassous brutus bait not me
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ill not endure it you forget yourself
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       to hedge me in i am a soldier i
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       older in practice abler than yourself
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       to make conditions
(observations): nwxfeaqxcwi gngwij
(states):       brutus go to you are not cassous
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       cassous i am
(observations): cejjgtjxgxef
(states):       brutus i say you are not
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       cassous urge me no more i shall forget myself
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 63; log likelihood -87382.0256
(states):       than such a roman
(observations): noeixjtcoxe

(states):       than such a roman
(observations): noeixjtcoxexhwfei
(states):       cassous brutus bait not me
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ill not endure it you forget yourself
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       to hedge me in i am a soldier i
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       older in practice abler than yourself
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       to make conditions
(observations): nwxfeaqxcwi gngwij
(states):       brutus go to you are not cassous
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       cassous i am
(observations): cejjgtjxgxef
(states):       brutus i say you are not
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       cassous urge me no more i shall forget myself
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 72; log likelihood -87369.3490
(states):       than such a roman
(observations): noeixjtcoxe

(states):       than such a roman
(observations): noeixjtcoxexhwfei
(states):       cassous brutus bait not me
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ill not endure it you forget yourself
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       to hedge me in i am a soldier i
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       older in practice abler than yourself
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       to make conditions
(observations): nwxfeaqxcwi gngwij
(states):       brutus go to you are not cassous
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       cassous i am
(observations): cejjgtjxgxef
(states):       brutus i say you are not
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       cassous urge me no more i shall forget myself
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 81; log likelihood -87358.9994
(states):       than such a roman
(observations): noeixjtcoxe

(states):       than such a roman
(observations): noeixjtcoxexhwfei
(states):       cassous brutus bait not me
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ill not endure it you forget yourself
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       to hedge me in i am a soldier i
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       older in practice abler than yourself
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       to make conditions
(observations): nwxfeaqxcwi gngwij
(states):       brutus go to you are not cassous
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       cassous i am
(observations): cejjgtjxgxef
(states):       brutus i say you are not
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       cassous urge me no more i shall forget myself
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 90; log likelihood -87349.5972
(states):       than such a roman
(observations): noeixjtcoxe

(states):       than such a roman
(observations): noeixjtcoxexhwfei
(states):       cassous brutus bait not me
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ill not endure it you forget yourself
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       to hedge me in i am a soldier i
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       older in practice abler than yourself
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       to make conditions
(observations): nwxfeaqxcwi gngwij
(states):       brutus go to you are not cassous
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       cassous i am
(observations): cejjgtjxgxef
(states):       brutus i say you are not
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       cassous urge me no more i shall forget myself
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 99; log likelihood -87343.8409


In [187]:
# Some data to use.
plaintext = 'plaintext/russian.txt'
ciphertext = 'encrypted/99_encrypted.txt' # longer sequences in russian                                                                                                     
char_to_int_mapping, int_to_char_mapping = get_char_to_int_mapping(plaintext)                                                                                                                                
plaintext_sequences = load_sequences(plaintext, char_to_int_mapping)
encrypted_sequences = load_sequences(ciphertext, char_to_int_mapping)
pi, A = estimate_markov_model_from_sequences(plaintext_sequences, len(char_to_int_mapping))
state_to_char_mapping = int_to_char_mapping
observation_to_char_mapping = int_to_char_mapping
hmm = HMM(observation_to_char_mapping, state_to_char_mapping)
hmm.estimate_with_em(encrypted_sequences, parameters={'pi': pi, 'A': A})

(states):       по  о  о   а а е о ао оа   о а о  о оо  е е о  о оо  о е о оо о   ео  оо о и   ооо     о о   е  е  о     ол оо оо о  о    о  а о о  а оо е е е  о  е  оо  а и  ое о   о оо о  о    о       но оо оо   о е ое    о    ое   но о а о о  а оо оо оо   о е ое    о    оа   оо     о с  о  о     е о о    о  но     о      е  е ао е оо о  а оо оо оо   о е ото  о е  е ао е о  но  р  о   а е о   о  о о    о  е  о   о       т  о   е   о ое о е  оо оо   о  но   о  ао е о  е   о ое  о    о  а   а о о оо но е  о оо   о о  о с  о ото  то  о    о  
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяръуо0ъпф о5мфювпъуошщн1ошщн1повоф 5юо о0ювмщгмддюодмю4 шпддюошфройдрьропдшщмроюхмоьпяфпйпф о оюяръуо0ъпф о5мфювпъу0роъю4моьпяфпйпфпойдрьцопдшщмцох2фюою3мв шдюодмфювйюодюошфрошвнто4мд8 дойпьпфю0уоъпйом0ъм0ъ

(states):       по ей  о о и а е о ао оро    а о ио оо  е е о по оо  о е о о  о и е   оо о и п ооо и   о о   е  е оо о   ой то о  о  о о  о  а о о  а ой е е е но  е  ой  а по ой ои ро о  о  о н  о  е  я ео то о    о ерой    о  ерой и но о а о о  а ой то о    о ерой н  о н  о  и оо     о и  о оо  о  е о оо   о  но е   о   в  е  е ао е о     а ой то о    о еро9о  о е  е ао е о  но ер  о   а е о е о е  о  о о се  о  оо  ое и е оо о е е о ой о е  оо во  оо  но е о  ао е о  е е о ой  о    о  а  оа о о о  но е  о оо   о о  о и  о ото  то  ое о о  
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяръуо0ъпф о5мфювпъуошщн1ошщн1повоф 5юо о0ювмщгмддюодмю4 шпддюошфройдрьропдшщмроюхмоьпяфпйпф о оюяръуо0ъпф о5мфювпъу0роъю4моьпяфпйпфпойдрьцопдшщмцох2фюою3мв шдюодмфювйюодюошфрошвнто4мд8 дойпьпфю0уоъпйом0ъм0ъ

(states):       повей ноно и а е о по оро    а о по ооа ере о по ота оде о о  о и е   то о и проро и с о о а е  е ко о а ой во о  но о ня о  а о о  авой ере е во  е вой  а порой виеро о  но о ня о  е  я е  то о  й о ерой в  оя ерой и е    а о о  авой во о  й о ерой но о ня о  е о й    о и оо оо  о  е о оо  оо  но е   оно в  е  е аоре о     авой во о  й о еройо  о е  е аоре о  то ер  оно а е о а о и ио но о ие  о  оо  ое и е ко о е е о ой о е  то вое оо  то а о  поре о  е е о ой  о    о  ая оа о оро  то е но по   о о  о и оо ойов ко  те о о  
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяръуо0ъпф о5мфювпъуошщн1ошщн1повоф 5юо о0ювмщгмддюодмю4 шпддюошфройдрьропдшщмроюхмоьпяфпйпф о оюяръуо0ъпф о5мфювпъу0роъю4моьпяфпйпфпойдрьцопдшщмцох2фюою3мв шдюодмфювйюодюошфрошвнто4мд8 дойпьпфю0уоъпйом0ъм0ъ

(states):       повый ноно ита еро по про а  а и по ота ере и порова одело о не ире а во ори промо и сторо а е не ко о а ой во о  ня е ия е наро и равой ере е во се вой ра порой виеро о  ня е ия е не ня е  толо  й о ерой ва ая ерой и е а  аро и равой воло  й о ерой во е ия е  е о и а  о и оо оо но ре о оо  вов еовев роно в  е не пореро а а авой воло  й о еройтано е не пореро  еовевароно а е о а ори ио но о ие но  вов ве и е ко о ереро ой о е  то вое ко  то а о  пореро нереро ой ро а  о сая ва о оро  то е ня по ало о  о и полойов ко  те оро  
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяръуо0ъпф о5мфювпъуошщн1ошщн1повоф 5юо о0ювмщгмддюодмю4 шпддюошфройдрьропдшщмроюхмоьпяфпйпф о оюяръуо0ъпф о5мфювпъу0роъю4моьпяфпйпфпойдрьцопдшщмцох2фюою3мв шдюодмфювйюодюошфрошвнто4мд8 дойпьпфю0уоъпйом0ъм0ъ

(states):       повый ново эть еро по при а на и по ованерени порованодело о ни ире а во ори промели сторо а е не ко о арая воло  ия е ия е на о и равый ере е во се вой ль порой виероло  ия е ия е не ня е  толо  и о ерой валая ирой оле а наро и равый воло  и о ерой во е ия е нето и а  о и оо оо со ре и оо свов аовев рово в  е не пореро а алавый воло  и о еройтано е не пореро саовиварово с е о а при ио но о 1е но свов ве и е ко оне еро ой оле  во вое оо сто ало  кореро не еро ой ро а но сья ва о еро  то е ни по ало о со и колойта ко сте оло  
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяръуо0ъпф о5мфювпъуошщн1ошщн1повоф 5юо о0ювмщгмддюодмю4 шпддюошфройдрьропдшщмроюхмоьпяфпйпф о оюяръуо0ъпф о5мфювпъу0роъю4моьпяфпйпфпойдрьцопдшщмцох2фюою3мв шдюодмфювйюодюошфрошвнто4мд8 дойпьпфю0уоъпйом0ъм0ъ

...

(states):       ая и блегогона прудаю при с нали пемуконазани пелакодужымору ни има с кумерись ужери онето сть серееме стох кому ней 1не0ть налолистаго назаны го нылок ста иело ские ому ней 14е0ть ни сята сколу сорушило с стоя ило нята с налолистаго сколу сорушило ней 1не0ть нену о с были перей не ть ерей нех ная и блегогость низающалу с стаго сколу сорушило коноль низающалу ная иодлего оныму с при ее неруший не нех ней усерееменалилу к нозав ко кой пй сто сте нающалу налилу к сте с не ная галуль у стоть нилью сте е ныли перо косто ски ере  
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяръуо0ъпф о5мфювпъуошщн1ошщн1повоф 5юо о0ювмщгмддюодмю4 шпддюошфройдрьропдшщмроюхмоьпяфпйпф о оюяръуо0ъпф о5мфювпъу0роъю4моьпяфпйпфпойдрьцопдшщмцох2фюою3мв шдюодмфювйюодюошфрошвнто4мд8 дойпьпфю0уоъпйом0ъм0ъ

(states):       ая и блегогона прудаю при с нали пемуконазани пелакодужемору ни има с кумерись ужери жнето сть серееме стох кому ней 1не0ть налолистаго назаны го нылок ста иело ские ому ней 14е0ть ни сята сколу сорушило с стоя ило нята с налолистаго сколу сорушило ней 1не0ть нену о с были перей не ть ерей нех ная и блегогость низающалу с стаго сколу сорушило коноль низающалу ная иодлего оныму с при ее неруший не нех ней усерееменалилу к нозав ко кой пе сто сте нающалу налилу к сте с не ная галуль у стоть нилью сте е ныли перо косто ски ере  
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяръуо0ъпф о5мфювпъуошщн1ошщн1повоф 5юо о0ювмщгмддюодмю4 шпддюошфройдрьропдшщмроюхмоьпяфпйпф о оюяръуо0ъпф о5мфювпъу0роъю4моьпяфпйпфпойдрьцопдшщмцох2фюою3мв шдюодмфювйюодюошфрошвнто4мд8 дойпьпфю0уоъпйом0ъм0ъ

(states):       ая и блегогона прудах при с нали пемуконазани пелакодужемору ни има с кумерись ужери жнето сть серееме стох кому ней 1не0ть налолистаго назаны го нылок ста иело ские ому ней 1не0ть ни сята сколу сорушило с стоя ило нята с налолистаго сколу сорушило ней 1не0ть нену о с были перей не ть ерей нех ная и блегогость низающалу с стаго сколу сорушило коноль низающалу ная иодлего оныму с при ее неруший не нех ней усерееменалилу к нозав ко кой пе сто сте нающалу налилу к сте с не ная галуль у столь нилью сте е ныли перо косто ски ере  
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяръуо0ъпф о5мфювпъуошщн1ошщн1повоф 5юо о0ювмщгмддюодмю4 шпддюошфройдрьропдшщмроюхмоьпяфпйпф о оюяръуо0ъпф о5мфювпъу0роъю4моьпяфпйпфпойдрьцопдшщмцох2фюою3мв шдюодмфювйюодюошфрошвнто4мд8 дойпьпфю0уоъпйом0ъм0ъ

(states):       ая и блегогона прудах при с нали пемуконазани пелакодужемору ни има с кумерись ужери жнето сть серееме стох кому ней 1не0ть налолистаго назаны го нылок ста иело ские ому ней 1не0ть ни сята сколу сорушило с стоя ило нята с налолистаго сколу сорушило ней 1не0ть нину о с были перей не ть ерей нех ная и блегогость низающалу с стаго сколу сорушило коноль низающалу ная иодлего оныму с при ее неруший не нех ней усерееменалилу к нозав ко кой пе сто сте нающалу налилу к сте с не ная галуль у столь нилью сте е ныли перо косто ски ере  
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяръуо0ъпф о5мфювпъуошщн1ошщн1повоф 5юо о0ювмщгмддюодмю4 шпддюошфройдрьропдшщмроюхмоьпяфпйпф о оюяръуо0ъпф о5мфювпъу0роъю4моьпяфпйпфпойдрьцопдшщмцох2фюою3мв шдюодмфювйюодюошфрошвнто4мд8 дойпьпфю0уоъпйом0ъм0ъ

(states):       ая и блегогона прудах при с нали пемуконазани пелакодужемору ни има с кумерись ужери жнето сть серееме стох кому ней 1не0ть налолистаго назаны го нылок ста иело ские ому ней 1не0ть ни сята сколу сорушило с стоя ило нята с налолистаго сколу сорушило ней 1не0ть нину о с были перей не ть ерей нех ная и блегогость низающалу с стаго сколу сорушило коноль низающалу ная иодлего оныму с при ее неруший не нех ней усерееменалилу к нозав ко кой пе сто сте нающалу налилу к сте с не ная галуль у столь нилью сте е ныли перо косто ски ере  
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяръуо0ъпф о5мфювпъуошщн1ошщн1повоф 5юо о0ювмщгмддюодмю4 шпддюошфройдрьропдшщмроюхмоьпяфпйпф о оюяръуо0ъпф о5мфювпъу0роъю4моьпяфпйпфпойдрьцопдшщмцох2фюою3мв шдюодмфювйюодюошфрошвнто4мд8 дойпьпфю0уоъпйом0ъм0ъ

**TASK 5**: Try to classify the author of each text. 

In [7]:
import os
import multiprocessing

In [21]:
def decrypt(arg):
    text = arg[0]
    int_to_char_mapping = arg[1]
    char_to_int_mapping = arg[2]
    if text[-4:] != '.txt':
        return
    ecnrypted_text = os.path.join('encrypted', text)
    encrypted_sequences = load_sequences(ecnrypted_text, char_to_int_mapping)
    prefloglike = -1000000
    best_model = None
    with open(os.path.join('logs', text), 'w') as fout:
        print(text + ':', file=fout)
        for i in range(10):
            hmm = HMM(int_to_char_mapping, int_to_char_mapping)
            loglikelihood = hmm.estimate_with_em(encrypted_sequences, parameters={'pi': pi, 'A': A}, max_iters=10, verbose=False)
            print('\t {} - {}'.format(i, loglikelihood), file=fout)
            if loglikelihood > prefloglike:
                prefloglike = loglikelihood
                best_model = hmm
        final = hmm.estimate_with_em(encrypted_sequences, parameters={'pi': pi, 'A': A}, max_iters=100, verbose=False)
        print('\t final - {}'.format(final), file=fout)
    with open(os.path.join('decrypted', text), 'w') as fout:
        for sequence in encrypted_sequences:
            print(''.join([int_to_char_mapping[x] for x in best_model.max_posterior_decode(sequence)]), file=fout)

In [10]:
plaintext = 'plaintext/all.txt'
char_to_int_mapping, int_to_char_mapping = get_char_to_int_mapping(plaintext)
plaintext_sequences = load_sequences(plaintext, char_to_int_mapping)
pi, A = estimate_markov_model_from_sequences(plaintext_sequences, len(char_to_int_mapping))
state_to_char_mapping = int_to_char_mapping
observation_to_char_mapping = int_to_char_mapping
encrypted_texts = os.listdir('encrypted')
data = []
for text in encrypted_texts:
    data.append((text, int_to_char_mapping, char_to_int_mapping))
pool = multiprocessing.Pool(12)
pool.map(decrypt, data)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [26]:
def find_poet(text):
    if text[-4:] != '.txt':
        return
    best_loglike = -1000000
    author = ''
    ecnrypted_text = os.path.join('encrypted', text)
    for poet in ['dickens.txt', 'evgeniy-onegin.txt', 'khlebnikov.txt', 'mayakovsky.txt', 'shakespeare.txt',
                 'voina-i-mir-tom-1.txt', 'voina-i-mir-tom-2.txt']:
        plaintext = os.path.join('plaintext', poet)
        char_to_int_mapping, int_to_char_mapping = get_char_to_int_mapping(plaintext)
        plaintext_sequences = load_sequences(plaintext, char_to_int_mapping)
        pi, A = estimate_markov_model_from_sequences(plaintext_sequences, len(char_to_int_mapping))
        try:
            encrypted_sequences = load_sequences(ecnrypted_text, char_to_int_mapping)
        except KeyError:
            continue
        hmm = HMM(int_to_char_mapping, int_to_char_mapping)
        loglikelihood = hmm.estimate_with_em(encrypted_sequences, parameters={'pi': pi, 'A': A}, max_iters=30, verbose=False)
        if loglikelihood > best_loglike:
            best_loglike = loglikelihood
            author = poet
    print(text, author)
    return text, author

In [27]:
plaintext = 'plaintext/all.txt'
encrypted_texts = os.listdir('encrypted')
data = []
for text in encrypted_texts:
    data.append(text)
pool = multiprocessing.Pool(12)
poets = pool.map(find_poet, data)

23_encrypted.txt dickens.txt
136_encrypted.txt dickens.txt
30_encrypted.txt dickens.txt
141_encrypted.txt dickens.txt
37_encrypted.txt dickens.txt
47_encrypted.txt dickens.txt
44_encrypted.txt shakespeare.txt
79_encrypted.txt dickens.txt
40_encrypted.txt dickens.txt
54_encrypted.txt dickens.txt
10_encrypted.txt shakespeare.txt
50_encrypted.txt shakespeare.txt
142_encrypted.txt shakespeare.txt
33_encrypted.txt shakespeare.txt
89_encrypted.txt shakespeare.txt
57_encrypted.txt shakespeare.txt
132_encrypted.txt shakespeare.txt
118_encrypted.txt shakespeare.txt
126_encrypted.txt shakespeare.txt
43_encrypted.txt shakespeare.txt
8_encrypted.txt dickens.txt
94_encrypted.txt dickens.txt
34_encrypted.txt dickens.txt
24_encrypted.txt dickens.txt
121_encrypted.txt dickens.txt
53_encrypted.txt dickens.txt
131_encrypted.txt dickens.txt
27_encrypted.txt evgeniy-onegin.txt
108_encrypted.txt khlebnikov.txt
125_encrypted.txt evgeniy-onegin.txt
29_encrypted.txt dickens.txt
138_encrypted.txt dickens.txt
7

In [33]:
data = [val[1] for val in sorted([(int(val[0].split('_')[0]), val) for val in poets])]

[('0_encrypted.txt', 'dickens.txt'),
 ('1_encrypted.txt', 'shakespeare.txt'),
 ('2_encrypted.txt', 'shakespeare.txt'),
 ('3_encrypted.txt', 'shakespeare.txt'),
 ('4_encrypted.txt', 'shakespeare.txt'),
 ('5_encrypted.txt', 'mayakovsky.txt'),
 ('6_encrypted.txt', 'shakespeare.txt'),
 ('7_encrypted.txt', 'shakespeare.txt'),
 ('8_encrypted.txt', 'dickens.txt'),
 ('9_encrypted.txt', 'shakespeare.txt'),
 ('10_encrypted.txt', 'shakespeare.txt'),
 ('11_encrypted.txt', 'voina-i-mir-tom-2.txt'),
 ('12_encrypted.txt', 'shakespeare.txt'),
 ('13_encrypted.txt', 'dickens.txt'),
 ('14_encrypted.txt', 'mayakovsky.txt'),
 ('15_encrypted.txt', 'shakespeare.txt'),
 ('16_encrypted.txt', 'shakespeare.txt'),
 ('17_encrypted.txt', 'evgeniy-onegin.txt'),
 ('18_encrypted.txt', 'dickens.txt'),
 ('19_encrypted.txt', 'evgeniy-onegin.txt'),
 ('20_encrypted.txt', 'dickens.txt'),
 ('21_encrypted.txt', 'dickens.txt'),
 ('22_encrypted.txt', 'shakespeare.txt'),
 ('23_encrypted.txt', 'dickens.txt'),
 ('24_encrypted.txt'

In [35]:
for line in data:
    print(line[0], line[1])

0_encrypted.txt dickens.txt
1_encrypted.txt shakespeare.txt
2_encrypted.txt shakespeare.txt
3_encrypted.txt shakespeare.txt
4_encrypted.txt shakespeare.txt
5_encrypted.txt mayakovsky.txt
6_encrypted.txt shakespeare.txt
7_encrypted.txt shakespeare.txt
8_encrypted.txt dickens.txt
9_encrypted.txt shakespeare.txt
10_encrypted.txt shakespeare.txt
11_encrypted.txt voina-i-mir-tom-2.txt
12_encrypted.txt shakespeare.txt
13_encrypted.txt dickens.txt
14_encrypted.txt mayakovsky.txt
15_encrypted.txt shakespeare.txt
16_encrypted.txt shakespeare.txt
17_encrypted.txt evgeniy-onegin.txt
18_encrypted.txt dickens.txt
19_encrypted.txt evgeniy-onegin.txt
20_encrypted.txt dickens.txt
21_encrypted.txt dickens.txt
22_encrypted.txt shakespeare.txt
23_encrypted.txt dickens.txt
24_encrypted.txt dickens.txt
25_encrypted.txt shakespeare.txt
26_encrypted.txt mayakovsky.txt
27_encrypted.txt evgeniy-onegin.txt
28_encrypted.txt shakespeare.txt
29_encrypted.txt dickens.txt
30_encrypted.txt dickens.txt
31_encrypted.tx